In [44]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [82]:
website = 'https://turbo.az/'

query_beginning = 'autos?page='

page_numbers = {"Mercedes": 250, "Bmw": 110, "Skoda": 4}

cars_query = {"Mercedes":"&q%5Bbarter%5D=0&q%5Bcrashed%5D=1&q%5Bcurrency%5D=azn&q%5Bengine_volume_from%5D=&q%5Bengine_volume_to%5D=&q%5Bfor_spare_parts%5D=0&q%5Bloan%5D=0&q%5Bmake%5D%5B%5D=4&q%5Bmileage_from%5D=&q%5Bmileage_to%5D=&q%5Bonly_shops%5D=&q%5Bpainted%5D=1&q%5Bpower_from%5D=&q%5Bpower_to%5D=&q%5Bprice_from%5D=&q%5Bprice_to%5D=&q%5Bregion%5D%5B%5D=&q%5Bsort%5D=&q%5Bused%5D=&q%5Byear_from%5D=&q%5Byear_to%5D=", 
        "Bmw":"&q%5Bbarter%5D=0&q%5Bcrashed%5D=1&q%5Bcurrency%5D=azn&q%5Bengine_volume_from%5D=&q%5Bengine_volume_to%5D=&q%5Bfor_spare_parts%5D=0&q%5Bloan%5D=0&q%5Bmake%5D%5B%5D=3&q%5Bmileage_from%5D=&q%5Bmileage_to%5D=&q%5Bonly_shops%5D=&q%5Bpainted%5D=1&q%5Bpower_from%5D=&q%5Bpower_to%5D=&q%5Bprice_from%5D=&q%5Bprice_to%5D=&q%5Bregion%5D%5B%5D=&q%5Bsort%5D=&q%5Bused%5D=&q%5Byear_from%5D=&q%5Byear_to%5D=",
        "Skoda":"&q%5Bbarter%5D=0&q%5Bcrashed%5D=1&q%5Bcurrency%5D=azn&q%5Bengine_volume_from%5D=&q%5Bengine_volume_to%5D=&q%5Bfor_spare_parts%5D=0&q%5Bloan%5D=0&q%5Bmake%5D%5B%5D=22&q%5Bmileage_from%5D=&q%5Bmileage_to%5D=&q%5Bonly_shops%5D=&q%5Bpainted%5D=1&q%5Bpower_from%5D=&q%5Bpower_to%5D=&q%5Bprice_from%5D=&q%5Bprice_to%5D=&q%5Bregion%5D%5B%5D=&q%5Bsort%5D=&q%5Bused%5D=&q%5Byear_from%5D=&q%5Byear_to%5D="}


In [ ]:
for j in range(len(cars_query)):
    for i in range(list(page_numbers.values())[j]):
        url = f'{website}{query_beginning}{i}{list(cars_query.values())[j]}'
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Safari/605.1.15'}
        print(url)
        response = requests.get(url, headers= headers)

        if response.status_code == 200:

            soup = BeautifulSoup(response.text, 'html.parser')

            model_names = [h2.text for h2 in soup.find_all('div', class_='products-i__name products-i__bottom-text')]
            model_years = [h2.text.split(',')[0] for h2 in soup.find_all('div', class_='products-i__attributes products-i__bottom-text')]
            photos = [img['src'] for img in soup.find_all('img')]

            photo_urls = []
            for img in soup.find_all('img'):
                if 'https://turbo.azstatic.com/uploads/' in img['src']:
                    photo_urls.append(img['src'])

            i = 1;
            for model_name, photo_url in zip(model_names, photo_urls):
                folder_path = os.path.join('output', model_name + '_' + model_years[i-1])

                # Check if the folder already exists
                if not os.path.exists(folder_path):
                    os.makedirs(folder_path)

                photo_filename = os.path.join(folder_path, f'{model_name}_{model_years[i-1]}photo_{i}.jpg')
                photo_url = urljoin(url, photo_url)
                photo_data = requests.get(photo_url).content
                i += 1

                with open(photo_filename, 'ab') as photo_file:
                    photo_file.write(photo_data)

print("Scraping is complete.")
